# Homework 3
# Basketball Analytics: Distilling and Summarizing Information 
### Eri Martinez
### TA: Sergio Rodriguez
### Fri: 12pm

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import subprocess as sp
import helper_basketball as h
import imp
imp.reload(h);
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist

## Preparing Data
To begin, the matrix X is created and saved in a pickle file titled 'allpatterns2016-17.pkl'. Also, information about teams and their players is loaded for later use in analysis of shooting patterns.

In [ ]:
## get all 2016-17 teams
params = {'LeagueID':'00','Season': '2016-17'}
teams = h.get_nba_data('commonTeamYears', params).set_index('TEAM_ID')
allteams = teams.loc[teams.MAX_YEAR=='2017'].index.values

## get all 2016-17 players
params = {'LeagueID':'00', 'Season': '2016-17', 'IsOnlyCurrentSeason': '0'}
players = h.get_nba_data('commonallplayers', params).set_index('PERSON_ID')
allplyrs = players.loc[players.TEAM_ID.isin(allteams)].index.values

In [ ]:
allshots = pickle.load(open('allshots2016-17.pkl', 'rb'))
allmade = allshots

In [ ]:
#Bins are created to divide the court in order to calculate the matrix X
player_ids = allmade.PlayerID.unique()
num_players = player_ids.size

nx = xedges.size - 1
ny = yedges.size - 1

xedges = (np.linspace(start=-25, stop=25, num=151, dtype=np.float)) * 12
yedges = (np.linspace(start= -4, stop=31, num=106, dtype=np.float)) * 12

all_counts = {}
all_smooth = {}

for i, one in enumerate(allmade.groupby('PlayerID')):
    
    pid, pdf = one
    
    tmp1, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=True, sigma=2)
    tmp2, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=False)
    
    all_smooth[pid] = tmp1.reshape(-1)
    all_counts[pid] = tmp2.reshape(-1)

In [ ]:
#Here the matrix X is downloaded into a pickle file
pickle.dump(np.stack(all_smooth.values()).T, open('allpatterns2016-17.pkl', 'wb'))
X = pickle.load(open('allpatterns2016-17.pkl', 'rb'))

# Problem 1


The following show the first player's shooting pattern and their estimated shooting patterns. Each have a different r 
value, as well as thier norm difference.

In [ ]:
def non_negative_marix_decomp(n_components,train_data):
    import sklearn.decomposition as skld
    model = skld.NMF(n_components=n_components, init='nndsvda', max_iter=500, random_state=0)
    W = model.fit_transform(train_data)
    H = model.components_
    nmf = (W,H)
    return(nmf)


In [ ]:
r = 5
W_5,H_5 = non_negative_marix_decomp(n_components = r,train_data = X)
X5_hat = np.matmul(W_5,H_5)
fig, ax = plt.subplots(1,2, figsize=(20,60))
h.plot_shotchart(X[:,2], xedges, yedges, ax=ax[0]) 
h.plot_shotchart(X5_hat[:,2], xedges, yedges, ax=ax[1])
ax[0].set_title('Original Shooting Pattern for Player 1')
ax[1].set_title('Estimated Shooting Pattern (r=5), Player 1')

In [ ]:
r = 5
W_5_2,H_5_2 = non_negative_marix_decomp(n_components = r,train_data = X)
X5_hat_2 = np.matmul(W_5_2,H_5_2)
fig, ax = plt.subplots(1,2, figsize=(20,60))
h.plot_shotchart(X[:,5], xedges, yedges, ax=ax[0]) 
h.plot_shotchart(X5_hat_2[:,5], xedges, yedges, ax=ax[1])
ax[0].set_title('Original Shooting Pattern for Player 2')
ax[1].set_title('Estimated Shooting Pattern (r=5), Player 2')

In [ ]:
#difference in norm for r=5
from numpy import linalg as LA # To compute Frobenious norm
LA.norm(X-np.matmul(W_5,H_5),'fro')

In [ ]:
r = 15
W_15,H_15 = non_negative_marix_decomp(n_components = r,train_data = X)
X15_hat = np.matmul(W_15,H_15)
fig, ax = plt.subplots(1,2, figsize=(20,60))
h.plot_shotchart(X[:,2], xedges, yedges, ax=ax[0]) 
h.plot_shotchart(X15_hat[:,2], xedges, yedges, ax=ax[1])
ax[0].set_title('Original Shooting Pattern for Player 1')
ax[1].set_title('Estimated Shooting Pattern (r=15) for Player 1')

In [ ]:
r = 15
W_15_2,H_15_2 = non_negative_marix_decomp(n_components = r,train_data = X)
X15_hat_2 = np.matmul(W_15_2,H_15_2)
fig, ax = plt.subplots(1,2, figsize=(20,60))
h.plot_shotchart(X[:,5], xedges, yedges, ax=ax[0]) 
h.plot_shotchart(X15_hat_2[:,5], xedges, yedges, ax=ax[1])
ax[0].set_title('Original Shooting Pattern for Player 2')
ax[1].set_title('Estimated Shooting Pattern (r=15) for Player 2')

In [ ]:
#difference in norm for r=15
from numpy import linalg as LA 
LA.norm(X-np.matmul(W_15,H_15),'fro')

After analyzing the two players and the differences in the actual shooting pattern vs. the estimated shooting pattern, we see that the larger the r value, the more accurate the shooting pattern tends to be.

Here, we use a for loop to compare the various values of r to see which one is better. The one with the greatest decrease in error will be the best option.

In [ ]:
from numpy import linalg as LA 
r_values = np.arange(10)+1
error_norm = []
for r in r_values:
    W,H = non_negative_marix_decomp(n_components = r,train_data = X)
    error_norm.append(LA.norm(X-np.matmul(W,H),'fro'))

In [ ]:
plt.ylabel('error_norm')
plt.xlabel('r values')
plt.plot(r_values,error_norm)

From this we are able to see that an r around 3 may be the best choice in order to obtain the best results. Any value above 5 or 6 seems to be too large.

## Problem 2

Next, we plot the correlation matrix of X to see which players are most correalted with each other.

In [ ]:
X_ = X[:,0:362].copy()
R = np.corrcoef(X_,rowvar=0)

In [ ]:
import seaborn as sns; sns.set()
plt.subplots(figsize=(15,20))
ax = sns.heatmap(R)

## Players with the lowest and highest similarities

The following code gives us the positions in the R correlation matrix with the highest and lowest similarities.

In [ ]:
highcorr=np.triu(R, k=1)
min_=np.min(highcorr[np.nonzero(highcorr)])
max_=np.max(highcorr)
min_num=np.where(highcorr == min_)
max_num=np.where(highcorr == max_)
print(min_num)
print(max_num)

#### Lowest Correlated Shooting Patterns
Next, we see get the names of the players with the lowest similarities.

In [ ]:
playerids=player_ids.tolist()
min_one=playerids[23]
min_one
minplayone=players.get_value(min_one,"DISPLAY_LAST_COMMA_FIRST")
print(minplayone)
min_two=playerids[251]
min_two
minplaytwo=players.get_value(min_two,"DISPLAY_LAST_COMMA_FIRST")
print(minplaytwo)

In [ ]:
fig, ax= plt.subplots(1,2, figsize=(25,70))
h.plot_shotchart(X[:,23], xedges, yedges, ax=ax[0])
h.plot_shotchart(X[:,251], xedges, yedges, ax=ax[1])
ax[0].set_title('Barton, Will Shooting Pattern')
ax[1].set_title('Muscala, Mike Shooting Pattern')


Here, we can see how different the two player's shooting patterns are. Will tends to shoot more where the hoop is, while Mike is more towards the outskirts of the shooting range.

#### Highest Correlated Shooting Patterns

In [ ]:
max_one=playerids[235]
maxplayone=players.get_value(max_one,"DISPLAY_LAST_COMMA_FIRST")
print(maxplayone)
max_two=playerids[283]
maxplaytwo=players.get_value(max_two,"DISPLAY_LAST_COMMA_FIRST")
print(maxplaytwo)

In [ ]:
fig, ax= plt.subplots(1,2, figsize=(25,70))
h.plot_shotchart(X[:,235], xedges, yedges, ax=ax[0])
h.plot_shotchart(X[:,283], xedges, yedges, ax=ax[1])
ax[0].set_title('Mejri, Salah Shooting Pattern')
ax[1].set_title('Randolph, Zach Shooting Pattern')

Here, we see that Salah and Zach have very similar shooting patterns, almost identical.

## Hierarchical Clustering

In [ ]:
Z = linkage(X_.T,method='complete',metric='correlation')

In [ ]:
plt.title('Hierarchical Clustering Dendrogram (truncated)')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=90.,
    leaf_font_size=3.,
)
plt.show()

In [ ]:
from scipy.cluster.hierarchy import fcluster
no_max_clust = 10
cluster_id = fcluster(Z,no_max_clust,criterion='maxclust')
cluster_id

In [ ]:
R_clust = np.corrcoef(X_test.T[np.argsort(cluster_id)])

In [ ]:
plt.subplots(figsize=(20,15))
ax = sns.heatmap(R_clust)

## Problem 3

The H columns tells us how important each of the basis vectors W are. Depending on the correlation between each player, we might be able to see which players are in which position based on the types of shots they made and where they made them from. Since certain players are assigned to specific positions, they are restricted to making shots from their respective position. Therefore, players with similar positions tend to have similar shooting patterns. Becaue the positions don't typically change much, the expected results can be like the ones mentioned in the paper. This can be seen in thhe hierarchical clustering and the way that the players are grouped together.

## Problem 4

We could use this information to see which players are better matched up with each other. Depending on their correlation and their shooting style, we can pair up specific players on the court in order to help the team win more games. We could also potentially help predict the players shooting style for a particular game. 